In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision import datasets, models, transforms

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

# Define the ResNet model
model = models.resnet50(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(2048, 512),
    nn.BatchNorm1d(512),  # Batch normalization layer
    nn.ReLU(inplace=True),  # activation function
    nn.Linear(512, 8),
    nn.Softmax(dim=1)
)

C:\Users\AS-GP\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\AS-GP\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.fc.parameters())
# Define the grid of hyperparameters
batch_sizes = [16, 32, 64]
learning_rates = [0.0001, 0.001, 0.1]
weight_decays = [0.0001, 0.001, 0.01]



In [4]:
num_epochs = 5

In [5]:
def train_and_evaluate(batch_size, learning_rate , weight_decay , train_loader, test_loader):
    optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate, weight_decay=weight_decay)
    for epoch in range(num_epochs):
        best_val_accuracy =0;
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train().to(device) #added to device
                data_loader = train_loader
            else:
                model.eval()
                data_loader = test_loader
            running_loss = 0.0
            running_corrects = 0
            

            
            for data, target in data_loader:
                data = data.to(device)
                target = target.to(device)

                output = model(data)
                loss = criterion(output, target)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(output, 1)
                running_loss += loss.item() * data.size(0)
                running_corrects += torch.sum(preds == target.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))

            if(phase == 'validation' and epoch_acc > best_val_accuracy):
                    best_val_accuracy = epoch_acc
            
    return best_val_accuracy
                
        


In [6]:
input_path = "lisat_gaze_data_v1/"

In [7]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(), #for parallel computations
        normalize
    ]),
}


image_datasets = {
    'train':
    datasets.ImageFolder(input_path + 'lisat_gaze_data_v1/train', data_transforms['train']),
    'validation':
    datasets.ImageFolder(input_path + 'lisat_gaze_data_v1/val', data_transforms['validation'])
}

In [8]:
# Perform the grid search
best_parameters = ()
best_accuracy = 0.0
for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        for weight_decay in weight_decays:
            print(f"Batch size: {batch_size}, Learning rate: {learning_rate}, Weight decay: {weight_decay}")
            train_loader = torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True)
            test_loader = torch.utils.data.DataLoader(image_datasets['validation'], batch_size=batch_size, shuffle=False)
            accuracy = train_and_evaluate(batch_size, learning_rate, weight_decay , train_loader , test_loader)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_parameters = (batch_size, learning_rate, weight_decay)
                print(f"Current Best accuracy: {best_accuracy}")
                print(f"Current Best parameters: {best_parameters}")

print(f" Final Best parameters: {best_parameters}")
print(f"Final Best accuracy: {best_accuracy}")      

Batch size: 16, Learning rate: 0.0001, Weight decay: 0.0001
Epoch 1/5
----------
train loss: 1.6663, acc: 0.6521
validation loss: 1.6406, acc: 0.6499
Epoch 2/5
----------
train loss: 1.5593, acc: 0.7443
validation loss: 1.6304, acc: 0.6528
Epoch 3/5
----------
train loss: 1.5211, acc: 0.7790
validation loss: 1.6317, acc: 0.6568
Epoch 4/5
----------
train loss: 1.5009, acc: 0.7971
validation loss: 1.5839, acc: 0.7041
Epoch 5/5
----------
train loss: 1.4846, acc: 0.8129
validation loss: 1.6006, acc: 0.6839
Current Best accuracy: 0.6839485690584819
Current Best parameters: (16, 0.0001, 0.0001)
Batch size: 16, Learning rate: 0.0001, Weight decay: 0.001
Epoch 1/5
----------
train loss: 1.4776, acc: 0.8210
validation loss: 1.5911, acc: 0.6939
Epoch 2/5
----------
train loss: 1.4716, acc: 0.8297
validation loss: 1.5975, acc: 0.6995
Epoch 3/5
----------
train loss: 1.4684, acc: 0.8363
validation loss: 1.6045, acc: 0.6898
Epoch 4/5
----------
train loss: 1.4646, acc: 0.8397
validation loss: 1.6

train loss: 1.9812, acc: 0.2918
validation loss: 1.8809, acc: 0.3930
Batch size: 32, Learning rate: 0.1, Weight decay: 0.001
Epoch 1/5
----------
train loss: 1.9748, acc: 0.2907
validation loss: 1.9277, acc: 0.3391
Epoch 2/5
----------
train loss: 1.9590, acc: 0.3033
validation loss: 1.8698, acc: 0.4224
Epoch 3/5
----------
train loss: 1.9558, acc: 0.3037
validation loss: 1.9860, acc: 0.2727
Epoch 4/5
----------
train loss: 1.9529, acc: 0.3060
validation loss: 1.9169, acc: 0.3457
Epoch 5/5
----------
train loss: 1.9544, acc: 0.3069
validation loss: 1.9269, acc: 0.3384
Batch size: 32, Learning rate: 0.1, Weight decay: 0.01
Epoch 1/5
----------
train loss: 1.9898, acc: 0.2674
validation loss: 1.9268, acc: 0.3683
Epoch 2/5
----------
train loss: 1.9920, acc: 0.2649
validation loss: 1.9690, acc: 0.2926
Epoch 3/5
----------
train loss: 1.9908, acc: 0.2686
validation loss: 2.0881, acc: 0.1197
Epoch 4/5
----------
train loss: 1.9905, acc: 0.2666
validation loss: 2.0139, acc: 0.2310
Epoch 5/5
